# Lab: Decorators

## Documentation redux

Suppose we want to document our function, but we want to customize the message. 
For instance, if we write:

```python
@document("Calling {function.__name__} with {args} and {kwargs}")
def print_string(thestring):
    print(thestring)
    
print_string("foo")
```

we will get the following output:

```
Calling print_string with ('foo',) and {}
foo
```

Write the `document` decorator, keeping in mind
- you will need 3 (!) levels of nested functions
- you can access the name of a function with the `__name__` attribute (e.g. `func.__name__`)

```python
from functools import wraps

def myfactory(...):
    def decorator(function):
        @wraps(function)
        def wrapper(*a, **kw):
            ...
            result = function(*a, **kw)
            ...
            return result
        return wrapper
    return decorator
```

In [ ]:
from functools import wraps

def document(msg):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(msg.format(function=func, args=args, kwargs=kwargs))
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [ ]:
@document("Calling {function.__name__}(*{args}, **{kwargs})")
def print_string(thestring):
    "This is my excellent docstring"
    print(thestring)

print_string(thestring='something else')

In [ ]:
the_decorator = document("Calling {function.__name__} with {args} and {kwargs}")
@the_decorator
def print_string(thestring):
    "This is my excellent docstring"
    print(thestring)

print_string(thestring="foo")

In [ ]:
help(print_string)

In [ ]:
print_string??

## Timing

Suppose we want to instrument some code in production so that it records its cumulative elapsed time:

```python
import time
timers = {}

@keeptime(timers, 'foo')
def sleep_a_while():
    time.sleep(0.2)

for i in range(10):
    sleep_a_while()
    
print(timers)
```

we would want to get an output similar to `{'foo': 2.02}`. 

Write the `keeptime` decorator, keeping in mind
- you can get the current time with `time.time()` (in seconds since 00:00 Jan 1, 1970)

In [11]:
from functools import wraps

def keeptime(timer_dict, timer_name):
#     if timer_name not in timer_dict:
#         timer_dict[timer_name] = 0
    timer_dict.setdefault(timer_name, 0)
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            try:
                return func(*args, **kwargs)
            finally:
                elapsed = time.time() - start
                timer_dict[timer_name] += elapsed
        return wrapper
    return decorator

In [12]:
import time
timers = {}

@keeptime(timers, 'foo')
def sleep_a_while():
    'Docstring'
    time.sleep(0.2)

for i in range(10):
    sleep_a_while()
    
print(timers)

{'foo': 2.003610372543335}


In [13]:
help(sleep_a_while)

Help on function sleep_a_while in module __main__:

sleep_a_while()
    Docstring



In [14]:
sleep_a_while

<function __main__.sleep_a_while()>

## These solutions were 'extra credit'

In [ ]:
def keeptime(timer_ctr, timer_name):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            try:
                return func(*args, **kwargs)
            finally:
                elapsed = time.time() - now
                timer_ctr[timer_name] += elapsed
        return wrapper
    return decorator

In [ ]:
import time
from collections import Counter
timers = Counter()

@keeptime(timers, 'foo')
@keeptime(timers, 'bar')
def sleep_a_while():
    time.sleep(0.2)

for i in range(10):
    sleep_a_while()
    
print(timers)

# Function attribute for elapsed time?

In [ ]:
def attr_keeptime(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        now = time.time()
        try:
            return func(*args, **kwargs)
        finally:
            elapsed = time.time() - now
            wrapper._timer += elapsed
    wrapper._timer = 0
    return wrapper


In [ ]:
import time

@attr_keeptime
def sleep_a_while():
    time.sleep(0.2)   
#sleep_a_while = attr_keeptime(sleep_a_while)

for i in range(10):
    sleep_a_while()
    
print(sleep_a_while._timer)